In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

In [2]:
bucket_name = 'nlp-spam' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)


ap-south-1


In [3]:
s3=boto3.resource('s3')
try: 
    if my_region=='us-east-2':
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})
    print('done bucket has been created')
except Exception as e:
    print('exception: ',e)

done bucket has been created


In [1]:
prefix = 'stock-prediction'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

NameError: name 'bucket_name' is not defined

In [2]:
import pandas as pd

In [6]:
!wget https://raw.githubusercontent.com/krishnaik06/Stock-Sentiment-Analysis/master/Data.csv 

--2021-02-16 08:40:49--  https://raw.githubusercontent.com/krishnaik06/Stock-Sentiment-Analysis/master/Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7651016 (7.3M) [text/plain]
Saving to: ‘Data.csv.2’

Data.csv.2          100%[===================>]   7.30M  --.-KB/s    in 0.07s   

2021-02-16 08:40:50 (97.7 MB/s) - ‘Data.csv.2’ saved [7651016/7651016]



In [3]:
!pip install gensim
import nltk
#nltk.download('all')

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


In [4]:
df=pd.read_csv('Data.csv',encoding = "ISO-8859-1")

In [5]:
df.isnull().sum()

Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    1
Top24    3
Top25    3
dtype: int64

In [6]:
train_df=df[df['Date'] < '20150101']
test_df=df[df['Date'] > '20141231']

In [7]:
x=train_df.iloc[:,2:27]
x.replace('[^a-zA-z]',' ',regex=True,inplace=True)

In [8]:
x_test=test_df.iloc[:,2:27]
x_test.replace('[^a-zA-z]',' ',regex=True,inplace=True)

In [9]:
x_test.shape

(378, 25)

In [10]:
ind=[str(i) for i in range(0,25)]

In [11]:
x.columns=ind
x_test.columns=ind


In [12]:
for index in ind:
    x[index]=x[index].str.lower()
    x_test[index]=x_test[index].str.lower()
    

In [13]:
x

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,...,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title
1,scorecard,the best lake scene,leader german sleaze inquiry,cheerio boyo,the main recommendations,has cubie killed fees,has cubie killed fees,has cubie killed fees,hopkins furious at foster s lack of hannibal...,has cubie killed fees,...,on the critical list,the timing of their lives,dear doctor,irish court halts ira man s extradition to nor...,burundi peace initiative fades after rebels re...,pe points the way forward to the ecb,campaigners keep up pressure on nazi war crime...,jane ratcliffe,yet more things you wouldn t know without the ...,millennium bug fails to bite
2,coventry caught on counter by flo,united s rivals on the road to rio,thatcher issues defence before trial by video,police help smith lay down the law at everton,tale of trautmann bears two more retellings,england on the rack,pakistan retaliate with call for video of walsh,cullinan continues his cape monopoly,mcgrath puts india out of their misery,blair witch bandwagon rolls on,...,south melbourne australia,necaxa mexico,real madrid spain,raja casablanca morocco,corinthians brazil,tony s pet project,al nassr saudi arabia,ideal holmes show,pinochet leaves hospital after tests,useful links
3,pilgrim knows how to progress,thatcher facing ban,mcilroy calls for irish fighting spirit,leicester bin stadium blueprint,united braced for mexican wave,auntie back in fashion even if the dress look...,shoaib appeal goes to the top,hussain hurt by shambles but lays blame on e...,england s decade of disasters,revenge is sweet for jubilant cronje,...,putin admits yeltsin quit to give him a head s...,bbc worst hit as digital tv begins to bite,how much can you pay for,christmas glitches,upending a table chopping a line and scoring ...,scientific evidence unreliable defence claims,fusco wins judicial review in extradition case,rebels thwart russian advance,blair orders shake up of failing nhs,lessons of law s hard heart
4,hitches and horlocks,beckham off but united survive,breast cancer screening,alan parker,guardian readers are you all whingers,hollywood beyond,ashes and diamonds,whingers a formidable minority,alan parker part two,thuggery toxins and ties,...,most everywhere udis,most wanted chloe lunettes,return of the cane completely off the agenda,from sleepy hollow to greeneland,blunkett outlines vision for over s,embattled dobson attacks play now pay later ...,doom and the dome,what is the north south divide,aitken released from jail,gone aloft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,brazil declares emergency after babies a...,scientists find big yr old viking settlem...,paris attacks belgian police arrest ninth sus...,wiretapping reveals communication between turk...,russia airstrikes hundreds of syrian civilans...,pakistan changes neutral position on syrian ci...,north korean diplomat arrested in south africa...,german teachers want mein kampf on syllabus,north korea slave force earns kim jong un regi...,yemeni forces preparing to attack saudi...,...,report on sexual exploitation and abuse by pea...,baby girl dies after x president s son s staff...,china smog sparks red alerts in cities,us planned east berlin s systematic destructi...,qatar world cup workers e

In [14]:
para=[]
for i in range(0,len(x.index)):
    para.append(' '.join(str(a) for a in x.iloc[i,0:25]))

In [15]:
len(para)

3975

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [18]:
wordnet=PorterStemmer()
review = [wordnet.stem(word) for word in para if not word in set(stopwords.words('english'))]

In [19]:
len(review)

3975

In [20]:

from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X = cv.fit_transform(review).toarray()

In [21]:
X.shape

(3975, 46506)

In [32]:
X_test.shape

(378, 46506)

In [23]:
X.shape

(3975, 46506)

In [24]:
X.shape

(3975, 46506)

In [25]:
# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X,train_df['Label'])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
## Predict for the Test Dataset
test_transform= []
for i in range(0,len(x_test.index)):
    test_transform.append(' '.join(str(a) for a in x_test.iloc[i,0:25]))


print(len(test_transform))
wordnet=PorterStemmer()
review_test = [wordnet.stem(word) for word in test_transform if not word in set(stopwords.words('english'))]
print(len(review_test))


378
378


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
X_test = cv.transform(review_test).toarray()
print(X_test.shape)


(378, 46506)


In [ ]:
X

In [28]:
predictions = randomclassifier.predict(X_test)

In [29]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [31]:


matrix=confusion_matrix(test_df['Label'],predictions)
print(matrix)
score=accuracy_score(test_df['Label'],predictions)
print(score)
report=classification_report(test_df['Label'],predictions)
print(report)



[[143  43]
 [ 17 175]]
0.8412698412698413
              precision    recall  f1-score   support

           0       0.89      0.77      0.83       186
           1       0.80      0.91      0.85       192

    accuracy                           0.84       378
   macro avg       0.85      0.84      0.84       378
weighted avg       0.85      0.84      0.84       378

